<a href="https://colab.research.google.com/github/Imtiazul-Islam/ai-models/blob/main/TTS/Ultimate-TTS-Colab-CPU-Only-Mode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title 🚀 **CELL 1: Installation & Setup**
#@markdown ### Complete Installation - Run This First!
#@markdown This cell installs all required packages and sets up the environment.

import subprocess
import sys
import os

print("╔" + "═"*58 + "╗")
print("║" + " 🎙️ PROFESSIONAL TTS STUDIO - INSTALLATION ".center(58) + "║")
print("║" + " ElevenLabs-Style Text-to-Speech System ".center(58) + "║")
print("╚" + "═"*58 + "╝")

# Core packages
PACKAGES = [
    "edge-tts",
    "gTTS",
    "pyttsx3",
    "unidecode",
    "num2words",
    "inflect",
    "pydub",
    "nest-asyncio",
    "ipywidgets",
    "phonetics",
    "langdetect",
    "clean-text",
    "mutagen",
]

print("\n📦 Installing packages...")
for pkg in PACKAGES:
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", pkg])
        print(f"   ✓ {pkg}")
    except Exception as e:
        print(f"   ⚠ {pkg} (optional, continuing...)")

# System dependencies
print("\n🔧 Installing system dependencies...")
subprocess.run(["apt-get", "update", "-qq"], capture_output=True)
subprocess.run(["apt-get", "install", "-y", "-qq", "ffmpeg", "espeak-ng"], capture_output=True)
print("   ✓ FFmpeg & eSpeak installed")

# Imports
print("\n📚 Loading libraries...")
import json
import time
import hashlib
import asyncio
import re
import pickle
from pathlib import Path
from datetime import datetime
from typing import List, Dict, Optional, Tuple, Any
from dataclasses import dataclass, field, asdict
from enum import Enum

import numpy as np
import edge_tts
import nest_asyncio
from gtts import gTTS
from unidecode import unidecode
from num2words import num2words
import inflect
from pydub import AudioSegment
from pydub.effects import normalize, speedup

import ipywidgets as widgets
from IPython.display import display, Audio, HTML, clear_output, FileLink

nest_asyncio.apply()

# Directory Structure
BASE_DIR = Path("/content/tts_studio")
DIRS = {
    'audio': BASE_DIR / "audio",
    'batch': BASE_DIR / "batch",
    'audiobooks': BASE_DIR / "audiobooks",
    'exports': BASE_DIR / "exports",
    'history': BASE_DIR / "history",
    'temp': BASE_DIR / "temp",
    'presets': BASE_DIR / "presets",
    'projects': BASE_DIR / "projects",
}

for name, path in DIRS.items():
    path.mkdir(parents=True, exist_ok=True)

print(f"   ✓ Created workspace at: {BASE_DIR}")

# Load Edge TTS Voices
print("\n🎤 Loading voice library...")
async def _load_voices():
    return await edge_tts.list_voices()

EDGE_VOICES = asyncio.get_event_loop().run_until_complete(_load_voices())

# Organize voices by various categories
VOICES_BY_LOCALE = {}
VOICES_BY_LANGUAGE = {}
VOICES_BY_GENDER = {'Male': [], 'Female': []}

for v in EDGE_VOICES:
    locale = v['Locale']
    lang = locale.split('-')[0]
    gender = v['Gender']

    VOICES_BY_LOCALE.setdefault(locale, []).append(v)
    VOICES_BY_LANGUAGE.setdefault(lang, []).append(v)
    VOICES_BY_GENDER[gender].append(v)

print(f"   ✓ Loaded {len(EDGE_VOICES)} voices")
print(f"   ✓ {len(VOICES_BY_LOCALE)} locales | {len(VOICES_BY_LANGUAGE)} languages")
print(f"   ✓ {len(VOICES_BY_GENDER['Male'])} male | {len(VOICES_BY_GENDER['Female'])} female")

print("\n" + "═"*60)
print("✅ INSTALLATION COMPLETE! Proceed to Cell 2.")
print("═"*60)

In [ ]:
#@title 🔧 **CELL 2: Professional TTS Engine & Presets**
#@markdown ### Core Engine with Content Creation Presets
#@markdown This cell defines all voice presets, content types, and the main engine.

# ═══════════════════════════════════════════════════════════════════════════════
# ENUMS & DATA CLASSES
# ═══════════════════════════════════════════════════════════════════════════════

class VoiceType(Enum):
    """Voice character types"""
    MALE_DEEP = "male_deep"
    MALE_NORMAL = "male_normal"
    MALE_SOFT = "male_soft"
    MALE_YOUNG = "male_young"
    MALE_ELDER = "male_elder"
    FEMALE_DEEP = "female_deep"
    FEMALE_NORMAL = "female_normal"
    FEMALE_SOFT = "female_soft"
    FEMALE_YOUNG = "female_young"
    FEMALE_ELDER = "female_elder"
    CHILD_BOY = "child_boy"
    CHILD_GIRL = "child_girl"
    NARRATOR = "narrator"
    NEWS_ANCHOR = "news_anchor"
    ASSISTANT = "assistant"


class ContentType(Enum):
    """Content creation presets"""
    YOUTUBE_VIDEO = "youtube"
    PODCAST = "podcast"
    AUDIOBOOK = "audiobook"
    DOCUMENTARY = "documentary"
    COMMERCIAL = "commercial"
    ELEARNING = "elearning"
    MEDITATION = "meditation"
    NEWS_BROADCAST = "news"
    GAMING = "gaming"
    IVR_PHONE = "ivr"
    SOCIAL_MEDIA = "social"
    PRESENTATION = "presentation"
    STORYTELLING = "storytelling"
    VOICEOVER = "voiceover"
    CUSTOM = "custom"


class SpeakingStyle(Enum):
    """Speaking style presets"""
    NATURAL = "natural"
    PROFESSIONAL = "professional"
    ENERGETIC = "energetic"
    CALM = "calm"
    DRAMATIC = "dramatic"
    EDUCATIONAL = "educational"
    FRIENDLY = "friendly"
    SERIOUS = "serious"
    CHEERFUL = "cheerful"
    EMPATHETIC = "empathetic"
    AUTHORITATIVE = "authoritative"
    CONVERSATIONAL = "conversational"


@dataclass
class VoicePreset:
    """Voice preset configuration"""
    name: str
    voice_id: str
    voice_type: VoiceType
    description: str
    speed: float = 1.0
    pitch: float = 1.0
    volume: float = 1.0
    locale: str = "en-US"
    gender: str = "Female"
    tags: List[str] = field(default_factory=list)


@dataclass
class ContentPreset:
    """Content type preset configuration"""
    name: str
    content_type: ContentType
    description: str
    recommended_voices: List[str]
    speed_range: Tuple[float, float] = (0.9, 1.1)
    default_speed: float = 1.0
    default_pitch: float = 1.0
    pause_between_sentences: int = 300  # ms
    intro_silence: int = 500  # ms
    outro_silence: int = 500  # ms
    normalize_audio: bool = True
    fade_in: int = 0  # ms
    fade_out: int = 0  # ms
    style: SpeakingStyle = SpeakingStyle.NATURAL
    tips: str = ""


@dataclass
class TTSSettings:
    """Complete TTS settings"""
    text: str
    voice_id: str = "en-US-JennyNeural"
    speed: float = 1.0
    pitch: float = 1.0
    volume: float = 1.0
    engine: str = "edge"
    language: str = "en-US"

    # Preprocessing
    preprocess_text: bool = True
    expand_abbreviations: bool = True
    convert_numbers: bool = True
    convert_time: bool = True

    # Audio effects
    normalize: bool = False
    fade_in_ms: int = 0
    fade_out_ms: int = 0
    trim_silence: bool = False
    silence_start_ms: int = 0
    silence_end_ms: int = 0

    # Output
    output_format: str = "mp3"
    sample_rate: int = 24000


# ═══════════════════════════════════════════════════════════════════════════════
# VOICE PRESETS DATABASE
# ═══════════════════════════════════════════════════════════════════════════════

VOICE_PRESETS = {
    # ═══════ MALE VOICES ═══════
    "male_deep_1": VoicePreset(
        name="Marcus (Deep Male)",
        voice_id="en-US-GuyNeural",
        voice_type=VoiceType.MALE_DEEP,
        description="Deep, authoritative male voice. Perfect for documentaries and serious content.",
        speed=0.95, pitch=0.9, gender="Male",
        tags=["deep", "authoritative", "documentary", "serious"]
    ),
    "male_deep_2": VoicePreset(
        name="James (Deep UK)",
        voice_id="en-GB-RyanNeural",
        voice_type=VoiceType.MALE_DEEP,
        description="Deep British male voice with gravitas.",
        speed=0.95, pitch=0.85, gender="Male", locale="en-GB",
        tags=["deep", "british", "formal", "elegant"]
    ),
    "male_normal_1": VoicePreset(
        name="Alex (Standard Male)",
        voice_id="en-US-DavisNeural",
        voice_type=VoiceType.MALE_NORMAL,
        description="Clear, professional male voice for general use.",
        speed=1.0, pitch=1.0, gender="Male",
        tags=["standard", "professional", "clear", "versatile"]
    ),
    "male_normal_2": VoicePreset(
        name="Oliver (UK Male)",
        voice_id="en-GB-ThomasNeural",
        voice_type=VoiceType.MALE_NORMAL,
        description="Professional British male voice.",
        speed=1.0, pitch=1.0, gender="Male", locale="en-GB",
        tags=["british", "professional", "clear"]
    ),
    "male_soft_1": VoicePreset(
        name="Ethan (Soft Male)",
        voice_id="en-US-JasonNeural",
        voice_type=VoiceType.MALE_SOFT,
        description="Soft, gentle male voice. Great for meditation and calm content.",
        speed=0.9, pitch=1.05, gender="Male",
        tags=["soft", "gentle", "calm", "meditation"]
    ),
    "male_young_1": VoicePreset(
        name="Tyler (Young Male)",
        voice_id="en-US-TonyNeural",
        voice_type=VoiceType.MALE_YOUNG,
        description="Youthful, energetic male voice for modern content.",
        speed=1.05, pitch=1.1, gender="Male",
        tags=["young", "energetic", "modern", "casual"]
    ),
    "male_elder_1": VoicePreset(
        name="Walter (Elder Male)",
        voice_id="en-US-DavisNeural",
        voice_type=VoiceType.MALE_ELDER,
        description="Mature, wise-sounding male voice.",
        speed=0.85, pitch=0.85, gender="Male",
        tags=["mature", "wise", "storytelling", "grandfather"]
    ),

    # ═══════ FEMALE VOICES ═══════
    "female_normal_1": VoicePreset(
        name="Sarah (Standard Female)",
        voice_id="en-US-JennyNeural",
        voice_type=VoiceType.FEMALE_NORMAL,
        description="Clear, professional female voice. Most versatile option.",
        speed=1.0, pitch=1.0, gender="Female",
        tags=["standard", "professional", "versatile", "clear"]
    ),
    "female_normal_2": VoicePreset(
        name="Emma (UK Female)",
        voice_id="en-GB-SoniaNeural",
        voice_type=VoiceType.FEMALE_NORMAL,
        description="Professional British female voice.",
        speed=1.0, pitch=1.0, gender="Female", locale="en-GB",
        tags=["british", "professional", "elegant"]
    ),
    "female_deep_1": VoicePreset(
        name="Victoria (Deep Female)",
        voice_id="en-US-AriaNeural",
        voice_type=VoiceType.FEMALE_DEEP,
        description="Deep, sophisticated female voice with authority.",
        speed=0.95, pitch=0.9, gender="Female",
        tags=["deep", "authoritative", "sophisticated", "serious"]
    ),
    "female_soft_1": VoicePreset(
        name="Luna (Soft Female)",
        voice_id="en-US-SaraNeural",
        voice_type=VoiceType.FEMALE_SOFT,
        description="Soft, soothing female voice. Perfect for meditation and ASMR.",
        speed=0.9, pitch=1.05, gender="Female",
        tags=["soft", "soothing", "meditation", "asmr", "gentle"]
    ),
    "female_soft_2": VoicePreset(
        name="Grace (Gentle UK)",
        voice_id="en-GB-LibbyNeural",
        voice_type=VoiceType.FEMALE_SOFT,
        description="Gentle British female voice.",
        speed=0.9, pitch=1.0, gender="Female", locale="en-GB",
        tags=["soft", "british", "gentle", "warm"]
    ),
    "female_young_1": VoicePreset(
        name="Zoe (Young Female)",
        voice_id="en-US-MichelleNeural",
        voice_type=VoiceType.FEMALE_YOUNG,
        description="Youthful, vibrant female voice for energetic content.",
        speed=1.05, pitch=1.1, gender="Female",
        tags=["young", "vibrant", "energetic", "modern"]
    ),
    "female_elder_1": VoicePreset(
        name="Margaret (Elder Female)",
        voice_id="en-US-JennyNeural",
        voice_type=VoiceType.FEMALE_ELDER,
        description="Warm, mature female voice with wisdom.",
        speed=0.85, pitch=0.9, gender="Female",
        tags=["mature", "warm", "grandmother", "storytelling"]
    ),

    # ═══════ SPECIAL VOICES ═══════
    "narrator_1": VoicePreset(
        name="Nathan (Narrator)",
        voice_id="en-US-GuyNeural",
        voice_type=VoiceType.NARRATOR,
        description="Professional narrator voice for audiobooks and documentaries.",
        speed=0.95, pitch=0.95, gender="Male",
        tags=["narrator", "audiobook", "documentary", "storytelling"]
    ),
    "narrator_2": VoicePreset(
        name="Nicole (Narrator)",
        voice_id="en-US-AriaNeural",
        voice_type=VoiceType.NARRATOR,
        description="Professional female narrator for audiobooks.",
        speed=0.95, pitch=1.0, gender="Female",
        tags=["narrator", "audiobook", "storytelling"]
    ),
    "news_anchor_1": VoicePreset(
        name="Anderson (News)",
        voice_id="en-US-DavisNeural",
        voice_type=VoiceType.NEWS_ANCHOR,
        description="Professional news anchor voice.",
        speed=1.0, pitch=1.0, gender="Male",
        tags=["news", "professional", "broadcast", "anchor"]
    ),
    "news_anchor_2": VoicePreset(
        name="Rachel (News)",
        voice_id="en-US-JennyNeural",
        voice_type=VoiceType.NEWS_ANCHOR,
        description="Professional female news anchor.",
        speed=1.0, pitch=1.0, gender="Female",
        tags=["news", "professional", "broadcast", "anchor"]
    ),
    "assistant_1": VoicePreset(
        name="Ava (Assistant)",
        voice_id="en-US-JennyNeural",
        voice_type=VoiceType.ASSISTANT,
        description="Friendly virtual assistant voice.",
        speed=1.0, pitch=1.05, gender="Female",
        tags=["assistant", "friendly", "helpful", "IVR"]
    ),
    "child_girl_1": VoicePreset(
        name="Sophie (Child Girl)",
        voice_id="en-US-AnaNeural",
        voice_type=VoiceType.CHILD_GIRL,
        description="Young girl voice for children's content.",
        speed=1.0, pitch=1.2, gender="Female",
        tags=["child", "young", "kids", "playful"]
    ),

    # ═══════ AUSTRALIAN VOICES ═══════
    "au_female_1": VoicePreset(
        name="Olivia (AU Female)",
        voice_id="en-AU-NatashaNeural",
        voice_type=VoiceType.FEMALE_NORMAL,
        description="Professional Australian female voice.",
        speed=1.0, pitch=1.0, gender="Female", locale="en-AU",
        tags=["australian", "professional", "clear"]
    ),
    "au_male_1": VoicePreset(
        name="Liam (AU Male)",
        voice_id="en-AU-WilliamNeural",
        voice_type=VoiceType.MALE_NORMAL,
        description="Professional Australian male voice.",
        speed=1.0, pitch=1.0, gender="Male", locale="en-AU",
        tags=["australian", "professional", "clear"]
    ),
}


# ═══════════════════════════════════════════════════════════════════════════════
# CONTENT PRESETS DATABASE
# ═══════════════════════════════════════════════════════════════════════════════

CONTENT_PRESETS = {
    ContentType.YOUTUBE_VIDEO: ContentPreset(
        name="YouTube Video",
        content_type=ContentType.YOUTUBE_VIDEO,
        description="Optimized for YouTube videos with clear, engaging narration.",
        recommended_voices=["female_normal_1", "male_normal_1", "female_young_1", "male_young_1"],
        speed_range=(0.95, 1.1),
        default_speed=1.0,
        pause_between_sentences=250,
        intro_silence=300,
        outro_silence=500,
        normalize_audio=True,
        style=SpeakingStyle.ENERGETIC,
        tips="Keep sentences short and punchy. Add emphasis on key points."
    ),

    ContentType.PODCAST: ContentPreset(
        name="Podcast",
        content_type=ContentType.PODCAST,
        description="Natural, conversational tone for podcast episodes.",
        recommended_voices=["male_normal_1", "female_normal_1", "male_soft_1", "female_soft_1"],
        speed_range=(0.9, 1.05),
        default_speed=0.95,
        pause_between_sentences=400,
        intro_silence=500,
        outro_silence=1000,
        normalize_audio=True,
        style=SpeakingStyle.CONVERSATIONAL,
        tips="Use natural pauses. Vary your pacing for engagement."
    ),

    ContentType.AUDIOBOOK: ContentPreset(
        name="Audiobook",
        content_type=ContentType.AUDIOBOOK,
        description="Professional audiobook narration with proper pacing.",
        recommended_voices=["narrator_1", "narrator_2", "male_deep_1", "female_deep_1"],
        speed_range=(0.85, 1.0),
        default_speed=0.92,
        pause_between_sentences=500,
        intro_silence=1000,
        outro_silence=1500,
        normalize_audio=True,
        fade_in=500,
        fade_out=1000,
        style=SpeakingStyle.DRAMATIC,
        tips="Use slower pacing. Allow pauses for dramatic effect."
    ),

    ContentType.DOCUMENTARY: ContentPreset(
        name="Documentary",
        content_type=ContentType.DOCUMENTARY,
        description="Authoritative narration for documentaries and educational content.",
        recommended_voices=["male_deep_1", "male_deep_2", "narrator_1", "female_deep_1"],
        speed_range=(0.85, 1.0),
        default_speed=0.9,
        pause_between_sentences=400,
        intro_silence=500,
        outro_silence=800,
        normalize_audio=True,
        style=SpeakingStyle.AUTHORITATIVE,
        tips="Maintain gravitas. Use measured, deliberate pacing."
    ),

    ContentType.COMMERCIAL: ContentPreset(
        name="Commercial / Advertisement",
        content_type=ContentType.COMMERCIAL,
        description="Upbeat, persuasive voice for commercials and ads.",
        recommended_voices=["female_young_1", "male_young_1", "female_normal_1", "male_normal_1"],
        speed_range=(1.0, 1.2),
        default_speed=1.05,
        pause_between_sentences=200,
        intro_silence=200,
        outro_silence=300,
        normalize_audio=True,
        style=SpeakingStyle.ENERGETIC,
        tips="Be enthusiastic! Keep it short and memorable."
    ),

    ContentType.ELEARNING: ContentPreset(
        name="E-Learning / Tutorial",
        content_type=ContentType.ELEARNING,
        description="Clear, educational tone for tutorials and courses.",
        recommended_voices=["female_normal_1", "male_normal_1", "assistant_1"],
        speed_range=(0.9, 1.05),
        default_speed=0.95,
        pause_between_sentences=350,
        intro_silence=500,
        outro_silence=700,
        normalize_audio=True,
        style=SpeakingStyle.EDUCATIONAL,
        tips="Speak clearly. Allow time for concepts to sink in."
    ),

    ContentType.MEDITATION: ContentPreset(
        name="Meditation / Relaxation",
        content_type=ContentType.MEDITATION,
        description="Calm, soothing voice for meditation and relaxation content.",
        recommended_voices=["female_soft_1", "male_soft_1", "female_soft_2"],
        speed_range=(0.75, 0.9),
        default_speed=0.8,
        pause_between_sentences=800,
        intro_silence=2000,
        outro_silence=3000,
        normalize_audio=True,
        fade_in=1000,
        fade_out=2000,
        style=SpeakingStyle.CALM,
        tips="Speak slowly and gently. Use longer pauses between sentences."
    ),

    ContentType.NEWS_BROADCAST: ContentPreset(
        name="News Broadcast",
        content_type=ContentType.NEWS_BROADCAST,
        description="Professional news delivery style.",
        recommended_voices=["news_anchor_1", "news_anchor_2", "male_normal_1", "female_normal_1"],
        speed_range=(1.0, 1.1),
        default_speed=1.02,
        pause_between_sentences=250,
        intro_silence=300,
        outro_silence=500,
        normalize_audio=True,
        style=SpeakingStyle.PROFESSIONAL,
        tips="Clear, authoritative delivery. Maintain consistent pace."
    ),

    ContentType.GAMING: ContentPreset(
        name="Gaming / Streaming",
        content_type=ContentType.GAMING,
        description="Energetic voice for gaming content and streams.",
        recommended_voices=["male_young_1", "female_young_1", "male_normal_1"],
        speed_range=(1.0, 1.15),
        default_speed=1.05,
        pause_between_sentences=200,
        intro_silence=200,
        outro_silence=300,
        normalize_audio=True,
        style=SpeakingStyle.ENERGETIC,
        tips="Keep energy high! Be expressive and fun."
    ),

    ContentType.IVR_PHONE: ContentPreset(
        name="IVR / Phone System",
        content_type=ContentType.IVR_PHONE,
        description="Professional voice for phone systems and IVR.",
        recommended_voices=["assistant_1", "female_normal_1", "male_normal_1"],
        speed_range=(0.9, 1.0),
        default_speed=0.95,
        pause_between_sentences=400,
        intro_silence=300,
        outro_silence=500,
        normalize_audio=True,
        style=SpeakingStyle.PROFESSIONAL,
        tips="Clear pronunciation. Slightly slower for phone quality."
    ),

    ContentType.SOCIAL_MEDIA: ContentPreset(
        name="Social Media (TikTok/Reels)",
        content_type=ContentType.SOCIAL_MEDIA,
        description="Quick, engaging voice for short-form social content.",
        recommended_voices=["female_young_1", "male_young_1", "child_girl_1"],
        speed_range=(1.05, 1.25),
        default_speed=1.1,
        pause_between_sentences=150,
        intro_silence=100,
        outro_silence=200,
        normalize_audio=True,
        style=SpeakingStyle.ENERGETIC,
        tips="Fast and punchy! Grab attention quickly."
    ),

    ContentType.PRESENTATION: ContentPreset(
        name="Presentation / Corporate",
        content_type=ContentType.PRESENTATION,
        description="Professional voice for business presentations.",
        recommended_voices=["female_normal_1", "male_normal_1", "female_deep_1", "male_deep_1"],
        speed_range=(0.9, 1.05),
        default_speed=0.95,
        pause_between_sentences=300,
        intro_silence=500,
        outro_silence=700,
        normalize_audio=True,
        style=SpeakingStyle.PROFESSIONAL,
        tips="Clear and professional. Allow pauses for slides."
    ),

    ContentType.STORYTELLING: ContentPreset(
        name="Storytelling / Fiction",
        content_type=ContentType.STORYTELLING,
        description="Expressive voice for fictional narratives.",
        recommended_voices=["narrator_1", "narrator_2", "male_elder_1", "female_elder_1"],
        speed_range=(0.85, 1.0),
        default_speed=0.9,
        pause_between_sentences=500,
        intro_silence=800,
        outro_silence=1200,
        normalize_audio=True,
        fade_in=500,
        fade_out=800,
        style=SpeakingStyle.DRAMATIC,
        tips="Use emotion and variation. Build tension with pauses."
    ),

    ContentType.VOICEOVER: ContentPreset(
        name="General Voiceover",
        content_type=ContentType.VOICEOVER,
        description="Versatile voiceover settings for general use.",
        recommended_voices=["female_normal_1", "male_normal_1"],
        speed_range=(0.9, 1.1),
        default_speed=1.0,
        pause_between_sentences=300,
        intro_silence=400,
        outro_silence=500,
        normalize_audio=True,
        style=SpeakingStyle.NATURAL,
        tips="Natural delivery. Match the tone to your content."
    ),

    ContentType.CUSTOM: ContentPreset(
        name="Custom Configuration",
        content_type=ContentType.CUSTOM,
        description="Fully customizable settings for any use case.",
        recommended_voices=list(VOICE_PRESETS.keys()),
        speed_range=(0.5, 2.0),
        default_speed=1.0,
        pause_between_sentences=300,
        intro_silence=0,
        outro_silence=0,
        normalize_audio=False,
        style=SpeakingStyle.NATURAL,
        tips="Configure all settings manually to your needs."
    ),
}


# ═══════════════════════════════════════════════════════════════════════════════
# SPEAKING STYLE CONFIGURATIONS
# ═══════════════════════════════════════════════════════════════════════════════

STYLE_CONFIGS = {
    SpeakingStyle.NATURAL: {
        "speed_mod": 0,
        "pitch_mod": 0,
        "description": "Natural, everyday speaking voice"
    },
    SpeakingStyle.PROFESSIONAL: {
        "speed_mod": -0.02,
        "pitch_mod": -0.05,
        "description": "Clear, business-appropriate tone"
    },
    SpeakingStyle.ENERGETIC: {
        "speed_mod": 0.05,
        "pitch_mod": 0.05,
        "description": "Upbeat, enthusiastic delivery"
    },
    SpeakingStyle.CALM: {
        "speed_mod": -0.15,
        "pitch_mod": -0.05,
        "description": "Slow, soothing pace"
    },
    SpeakingStyle.DRAMATIC: {
        "speed_mod": -0.08,
        "pitch_mod": 0,
        "description": "Expressive, theatrical style"
    },
    SpeakingStyle.EDUCATIONAL: {
        "speed_mod": -0.05,
        "pitch_mod": 0,
        "description": "Clear, instructional tone"
    },
    SpeakingStyle.FRIENDLY: {
        "speed_mod": 0,
        "pitch_mod": 0.05,
        "description": "Warm, approachable voice"
    },
    SpeakingStyle.SERIOUS: {
        "speed_mod": -0.05,
        "pitch_mod": -0.08,
        "description": "Grave, important tone"
    },
    SpeakingStyle.CHEERFUL: {
        "speed_mod": 0.05,
        "pitch_mod": 0.08,
        "description": "Happy, positive delivery"
    },
    SpeakingStyle.EMPATHETIC: {
        "speed_mod": -0.05,
        "pitch_mod": 0,
        "description": "Understanding, compassionate voice"
    },
    SpeakingStyle.AUTHORITATIVE: {
        "speed_mod": -0.05,
        "pitch_mod": -0.1,
        "description": "Commanding, confident tone"
    },
    SpeakingStyle.CONVERSATIONAL: {
        "speed_mod": 0,
        "pitch_mod": 0.02,
        "description": "Natural, chat-like delivery"
    },
}


# ═══════════════════════════════════════════════════════════════════════════════
# TEXT PREPROCESSOR
# ═══════════════════════════════════════════════════════════════════════════════

class TextPreprocessor:
    """Advanced text preprocessing for TTS"""

    def __init__(self):
        self.inflect_engine = inflect.engine()

        self.abbreviations = {
            "mr.": "mister", "mrs.": "missus", "ms.": "miss", "dr.": "doctor",
            "prof.": "professor", "sr.": "senior", "jr.": "junior",
            "vs.": "versus", "etc.": "et cetera", "e.g.": "for example",
            "i.e.": "that is", "st.": "street", "ave.": "avenue",
            "blvd.": "boulevard", "apt.": "apartment", "govt.": "government",
            "dept.": "department", "univ.": "university", "corp.": "corporation",
            "inc.": "incorporated", "ltd.": "limited", "co.": "company",
            "jan.": "january", "feb.": "february", "mar.": "march",
            "apr.": "april", "jun.": "june", "jul.": "july", "aug.": "august",
            "sep.": "september", "oct.": "october", "nov.": "november",
            "dec.": "december", "mon.": "monday", "tue.": "tuesday",
            "wed.": "wednesday", "thu.": "thursday", "fri.": "friday",
            "sat.": "saturday", "sun.": "sunday",
        }

        self.symbols = {
            "&": " and ", "@": " at ", "#": " hashtag ", "%": " percent ",
            "+": " plus ", "=": " equals ", "°": " degrees ", "$": " dollars ",
            "€": " euros ", "£": " pounds ", "©": " copyright ",
        }

    def expand_abbreviations(self, text: str) -> str:
        for abbr, full in self.abbreviations.items():
            text = re.sub(re.escape(abbr), full, text, flags=re.IGNORECASE)
        return text

    def convert_numbers(self, text: str) -> str:
        def replace_num(match):
            num_str = match.group()
            try:
                if '.' in num_str:
                    parts = num_str.split('.')
                    whole = num2words(int(parts[0])) if parts[0] else "zero"
                    decimal = ' point ' + ' '.join([num2words(int(d)) for d in parts[1]])
                    return whole + decimal
                return num2words(int(num_str))
            except:
                return num_str
        return re.sub(r'\b\d+\.?\d*\b', replace_num, text)

    def convert_time(self, text: str) -> str:
        def time_to_words(match):
            hour, minute = int(match.group(1)), int(match.group(2))
            period = match.group(3) or ""
            hour_word = num2words(hour)
            minute_word = "o'clock" if minute == 0 else (f"oh {num2words(minute)}" if minute < 10 else num2words(minute))
            return f"{hour_word} {minute_word} {period}".strip()
        return re.sub(r'(\d{1,2}):(\d{2})\s*(AM|PM|am|pm)?', time_to_words, text)

    def convert_ordinals(self, text: str) -> str:
        def replace_ordinal(match):
            return self.inflect_engine.ordinal(int(match.group(1)))
        return re.sub(r'\b(\d+)(st|nd|rd|th)\b', replace_ordinal, text, flags=re.IGNORECASE)

    def replace_symbols(self, text: str) -> str:
        for symbol, word in self.symbols.items():
            text = text.replace(symbol, word)
        return text

    def clean_text(self, text: str) -> str:
        text = ' '.join(text.split())
        text = re.sub(r'([.!?])\1+', r'\1', text)
        text = re.sub(r'([.!?,])([A-Za-z])', r'\1 \2', text)
        return text.strip()

    def process(self, text: str, settings: TTSSettings = None) -> str:
        if settings is None or settings.expand_abbreviations:
            text = self.expand_abbreviations(text)
        if settings is None or settings.convert_numbers:
            text = self.convert_ordinals(text)
            text = self.convert_numbers(text)
        if settings is None or settings.convert_time:
            text = self.convert_time(text)
        text = self.replace_symbols(text)
        text = self.clean_text(text)
        return text


# ═══════════════════════════════════════════════════════════════════════════════
# AUDIO PROCESSOR
# ═══════════════════════════════════════════════════════════════════════════════

class AudioProcessor:
    """Audio processing and effects"""

    def __init__(self):
        self.temp_dir = DIRS['temp']

    def load_audio(self, file_path: str) -> AudioSegment:
        return AudioSegment.from_file(file_path)

    def save_audio(self, audio: AudioSegment, file_path: str, format: str = 'mp3') -> str:
        audio.export(file_path, format=format)
        return file_path

    def change_speed(self, audio: AudioSegment, speed: float) -> AudioSegment:
        if speed == 1.0:
            return audio
        if speed > 1.0:
            return speedup(audio, speed, 150, 25)
        new_frame_rate = int(audio.frame_rate * speed)
        return audio._spawn(audio.raw_data, overrides={'frame_rate': new_frame_rate}).set_frame_rate(audio.frame_rate)

    def normalize_audio(self, audio: AudioSegment) -> AudioSegment:
        return normalize(audio)

    def fade_in(self, audio: AudioSegment, duration_ms: int) -> AudioSegment:
        return audio.fade_in(duration_ms) if duration_ms > 0 else audio

    def fade_out(self, audio: AudioSegment, duration_ms: int) -> AudioSegment:
        return audio.fade_out(duration_ms) if duration_ms > 0 else audio

    def add_silence(self, audio: AudioSegment, start_ms: int = 0, end_ms: int = 0) -> AudioSegment:
        if start_ms > 0:
            audio = AudioSegment.silent(duration=start_ms) + audio
        if end_ms > 0:
            audio = audio + AudioSegment.silent(duration=end_ms)
        return audio

    def trim_silence(self, audio: AudioSegment, threshold: int = -50) -> AudioSegment:
        chunk_size = 10
        start_trim = end_trim = 0

        for i in range(0, len(audio), chunk_size):
            if audio[i:i+chunk_size].dBFS > threshold:
                start_trim = i
                break

        for i in range(len(audio), 0, -chunk_size):
            if audio[i-chunk_size:i].dBFS > threshold:
                end_trim = i
                break

        return audio[start_trim:end_trim] if end_trim > start_trim else audio

    def concatenate(self, audio_list: List[AudioSegment], gap_ms: int = 0) -> AudioSegment:
        if not audio_list:
            return AudioSegment.empty()

        result = audio_list[0]
        silence = AudioSegment.silent(duration=gap_ms) if gap_ms > 0 else AudioSegment.empty()

        for audio in audio_list[1:]:
            result = result + silence + audio

        return result

    def get_info(self, audio: AudioSegment) -> Dict:
        return {
            'duration_ms': len(audio),
            'duration_s': round(len(audio) / 1000, 2),
            'channels': audio.channels,
            'frame_rate': audio.frame_rate,
            'dBFS': round(audio.dBFS, 2)
        }

    def apply_effects(self, audio: AudioSegment, settings: TTSSettings) -> AudioSegment:
        if settings.normalize:
            audio = self.normalize_audio(audio)
        if settings.fade_in_ms > 0:
            audio = self.fade_in(audio, settings.fade_in_ms)
        if settings.fade_out_ms > 0:
            audio = self.fade_out(audio, settings.fade_out_ms)
        if settings.trim_silence:
            audio = self.trim_silence(audio)
        if settings.silence_start_ms > 0 or settings.silence_end_ms > 0:
            audio = self.add_silence(audio, settings.silence_start_ms, settings.silence_end_ms)
        return audio


# ═══════════════════════════════════════════════════════════════════════════════
# MAIN TTS ENGINE
# ═══════════════════════════════════════════════════════════════════════════════

class ProfessionalTTSEngine:
    """Professional TTS Engine with all capabilities"""

    def __init__(self):
        self.preprocessor = TextPreprocessor()
        self.audio_processor = AudioProcessor()
        self.history = []
        self.max_history = 500

    # ═══════ CORE SYNTHESIS METHODS ═══════

    async def _edge_tts_async(self, text: str, voice: str, rate: float, pitch: float, volume: float, output_file: str) -> str:
        rate_str = f"+{int((rate-1)*100)}%" if rate >= 1 else f"-{int((1-rate)*100)}%"
        pitch_str = f"+{int((pitch-1)*50)}Hz" if pitch >= 1 else f"-{int((1-pitch)*50)}Hz"
        volume_str = f"+{int((volume-1)*100)}%" if volume >= 1 else f"-{int((1-volume)*100)}%"

        communicate = edge_tts.Communicate(text, voice, rate=rate_str, pitch=pitch_str, volume=volume_str)
        await communicate.save(output_file)
        return output_file

    def edge_tts(self, text: str, voice: str, speed: float, pitch: float, volume: float, output_file: str) -> str:
        loop = asyncio.get_event_loop()
        loop.run_until_complete(self._edge_tts_async(text, voice, speed, pitch, volume, output_file))
        return output_file

    def google_tts(self, text: str, language: str, slow: bool, output_file: str) -> str:
        tts = gTTS(text=text, lang=language.split('-')[0], slow=slow)
        tts.save(output_file)
        return output_file

    # ═══════ MAIN SYNTHESIS ═══════

    def synthesize(self, settings: TTSSettings, progress_callback=None) -> Dict:
        """Main synthesis method with full settings"""
        start_time = time.time()

        # Preprocess text
        text = settings.text
        if settings.preprocess_text:
            text = self.preprocessor.process(text, settings)

        # Generate filename
        timestamp = int(time.time() * 1000)
        output_file = str(DIRS['audio'] / f"tts_{timestamp}.{settings.output_format}")

        # Generate audio
        if settings.engine == 'edge':
            self.edge_tts(text, settings.voice_id, settings.speed, settings.pitch, settings.volume, output_file)
        elif settings.engine == 'google':
            self.google_tts(text, settings.language, settings.speed < 0.9, output_file)

        # Apply audio effects
        if any([settings.normalize, settings.fade_in_ms, settings.fade_out_ms,
                settings.trim_silence, settings.silence_start_ms, settings.silence_end_ms]):
            audio = self.audio_processor.load_audio(output_file)
            audio = self.audio_processor.apply_effects(audio, settings)
            self.audio_processor.save_audio(audio, output_file, settings.output_format)

        # Get audio info
        audio = self.audio_processor.load_audio(output_file)
        info = self.audio_processor.get_info(audio)

        # Create result
        result = {
            'output_file': output_file,
            'duration_s': info['duration_s'],
            'processing_time': round(time.time() - start_time, 2),
            'settings': asdict(settings),
            'timestamp': datetime.now().isoformat()
        }

        # Save to history
        self.history.append(result)
        if len(self.history) > self.max_history:
            self.history = self.history[-self.max_history:]

        return result

    def synthesize_with_preset(self, text: str, voice_preset_id: str,
                                content_type: ContentType = ContentType.CUSTOM,
                                style: SpeakingStyle = None,
                                custom_overrides: Dict = None) -> Dict:
        """Synthesize using presets"""

        # Get voice preset
        voice_preset = VOICE_PRESETS.get(voice_preset_id)
        if not voice_preset:
            raise ValueError(f"Voice preset '{voice_preset_id}' not found")

        # Get content preset
        content_preset = CONTENT_PRESETS.get(content_type, CONTENT_PRESETS[ContentType.CUSTOM])

        # Get style config
        style = style or content_preset.style
        style_config = STYLE_CONFIGS.get(style, STYLE_CONFIGS[SpeakingStyle.NATURAL])

        # Build settings
        settings = TTSSettings(
            text=text,
            voice_id=voice_preset.voice_id,
            speed=voice_preset.speed + style_config['speed_mod'],
            pitch=voice_preset.pitch + style_config['pitch_mod'],
            volume=voice_preset.volume,
            engine="edge",
            language=voice_preset.locale,
            normalize=content_preset.normalize_audio,
            fade_in_ms=content_preset.fade_in,
            fade_out_ms=content_preset.fade_out,
            silence_start_ms=content_preset.intro_silence,
            silence_end_ms=content_preset.outro_silence,
        )

        # Apply custom overrides
        if custom_overrides:
            for key, value in custom_overrides.items():
                if hasattr(settings, key):
                    setattr(settings, key, value)

        return self.synthesize(settings)

    def synthesize_multi_voice(self, segments: List[Dict], gap_ms: int = 300) -> Dict:
        """Synthesize with multiple voices"""
        audio_segments = []

        for seg in segments:
            settings = TTSSettings(
                text=seg['text'],
                voice_id=seg.get('voice_id', 'en-US-JennyNeural'),
                speed=seg.get('speed', 1.0),
                pitch=seg.get('pitch', 1.0),
            )
            result = self.synthesize(settings)
            audio_segments.append(self.audio_processor.load_audio(result['output_file']))

        # Concatenate
        final_audio = self.audio_processor.concatenate(audio_segments, gap_ms)

        output_file = str(DIRS['audio'] / f"multi_{int(time.time())}.mp3")
        self.audio_processor.save_audio(final_audio, output_file)

        info = self.audio_processor.get_info(final_audio)

        return {
            'output_file': output_file,
            'duration_s': info['duration_s'],
            'segments': len(segments)
        }

    def batch_synthesize(self, texts: List[str], settings_template: TTSSettings,
                         merge: bool = False, gap_ms: int = 500,
                         progress_callback=None) -> Dict:
        """Batch synthesis"""
        results = []

        for i, text in enumerate(texts):
            settings = TTSSettings(**asdict(settings_template))
            settings.text = text
            result = self.synthesize(settings)
            results.append(result)

            if progress_callback:
                progress_callback(i + 1, len(texts))

        if merge and len(results) > 1:
            audio_segments = [self.audio_processor.load_audio(r['output_file']) for r in results]
            merged = self.audio_processor.concatenate(audio_segments, gap_ms)

            merged_file = str(DIRS['batch'] / f"batch_{int(time.time())}.mp3")
            self.audio_processor.save_audio(merged, merged_file)

            return {
                'merged_file': merged_file,
                'individual_files': [r['output_file'] for r in results],
                'total_duration_s': self.audio_processor.get_info(merged)['duration_s']
            }

        return {'files': results}


# Initialize engine
tts_engine = ProfessionalTTSEngine()

print("╔" + "═"*58 + "╗")
print("║" + " ✅ PROFESSIONAL TTS ENGINE READY ".center(58) + "║")
print("╚" + "═"*58 + "╝")
print(f"""
📊 Available Resources:
   • {len(VOICE_PRESETS)} Voice Presets (Male, Female, Child, Elder, etc.)
   • {len(CONTENT_PRESETS)} Content Type Presets
   • {len(STYLE_CONFIGS)} Speaking Styles
   • {len(EDGE_VOICES)} Edge TTS Voices

🎭 Voice Types: Deep, Normal, Soft, Young, Elder, Child, Narrator, News Anchor
📹 Content Types: YouTube, Podcast, Audiobook, Documentary, Commercial, E-Learning, etc.
🎨 Styles: Natural, Professional, Energetic, Calm, Dramatic, Educational, etc.
""")

In [ ]:
#@title 🎙️ **CELL 3: Professional TTS Studio Interface**
#@markdown ### ElevenLabs-Style Professional Interface
#@markdown Complete control over all TTS parameters with presets and custom configuration.

# ═══════════════════════════════════════════════════════════════════════════════
# HELPER FUNCTIONS
# ═══════════════════════════════════════════════════════════════════════════════

def get_voice_options_by_type(voice_type: str = 'all') -> List[Tuple[str, str]]:
    """Get voice options filtered by type"""
    options = []
    for preset_id, preset in VOICE_PRESETS.items():
        if voice_type == 'all' or preset.voice_type.value == voice_type:
            emoji = "👩" if preset.gender == "Female" else "👨"
            if "child" in preset.voice_type.value:
                emoji = "👧" if preset.gender == "Female" else "👦"
            elif "elder" in preset.voice_type.value:
                emoji = "👵" if preset.gender == "Female" else "👴"
            options.append((f"{emoji} {preset.name}", preset_id))
    return options

def get_content_options() -> List[Tuple[str, str]]:
    """Get content type options"""
    icons = {
        'youtube': '📺', 'podcast': '🎙️', 'audiobook': '📚', 'documentary': '🎬',
        'commercial': '📢', 'elearning': '🎓', 'meditation': '🧘', 'news': '📰',
        'gaming': '🎮', 'ivr': '📞', 'social': '📱', 'presentation': '📊',
        'storytelling': '📖', 'voiceover': '🎤', 'custom': '⚙️'
    }
    return [(f"{icons.get(ct.value, '🎯')} {preset.name}", ct.value)
            for ct, preset in CONTENT_PRESETS.items()]

def get_style_options() -> List[Tuple[str, str]]:
    """Get speaking style options"""
    icons = {
        'natural': '🌿', 'professional': '💼', 'energetic': '⚡', 'calm': '😌',
        'dramatic': '🎭', 'educational': '📚', 'friendly': '😊', 'serious': '😐',
        'cheerful': '😄', 'empathetic': '💕', 'authoritative': '👔', 'conversational': '💬'
    }
    return [(f"{icons.get(style.value, '🎯')} {style.value.title()}", style.value)
            for style in SpeakingStyle]


# ═══════════════════════════════════════════════════════════════════════════════
# MAIN UI COMPONENTS
# ═══════════════════════════════════════════════════════════════════════════════

# Header
header_html = widgets.HTML("""
<div style="background: linear-gradient(135deg, #1a1a2e 0%, #16213e 50%, #0f3460 100%);
            padding: 30px; border-radius: 20px; margin-bottom: 20px;
            box-shadow: 0 10px 40px rgba(0,0,0,0.3);">
    <h1 style="color: #e94560; margin: 0; font-size: 2.5em; text-align: center;
               text-shadow: 2px 2px 4px rgba(0,0,0,0.3);">
        🎙️ Professional TTS Studio
    </h1>
    <p style="color: #a2d2ff; margin: 10px 0 0 0; text-align: center; font-size: 1.1em;">
        ElevenLabs-Style Text-to-Speech • Unlimited Generation • CPU Optimized
    </p>
    <div style="display: flex; justify-content: center; gap: 20px; margin-top: 15px;">
        <span style="background: #e94560; color: white; padding: 5px 15px; border-radius: 20px; font-size: 0.85em;">
            🎭 {voice_count} Voice Presets
        </span>
        <span style="background: #0f3460; color: white; padding: 5px 15px; border-radius: 20px; font-size: 0.85em;">
            📹 {content_count} Content Types
        </span>
        <span style="background: #533483; color: white; padding: 5px 15px; border-radius: 20px; font-size: 0.85em;">
            🎨 {style_count} Speaking Styles
        </span>
    </div>
</div>
""".format(voice_count=len(VOICE_PRESETS), content_count=len(CONTENT_PRESETS), style_count=len(STYLE_CONFIGS)))


# ═══════════════════════════════════════════════════════════════════════════════
# TAB 1: QUICK GENERATE (Simple Mode)
# ═══════════════════════════════════════════════════════════════════════════════

quick_text = widgets.Textarea(
    value='Welcome to the Professional TTS Studio! This is your one-stop solution for creating high-quality voiceovers, podcasts, audiobooks, and more.',
    placeholder='Enter your text here...',
    layout=widgets.Layout(width='100%', height='180px')
)

quick_content_type = widgets.Dropdown(
    options=get_content_options(),
    value='youtube',
    description='Content:',
    style={'description_width': '80px'},
    layout=widgets.Layout(width='49%')
)

quick_voice_type = widgets.Dropdown(
    options=[
        ('🎭 All Voice Types', 'all'),
        ('👨 Male - Deep', 'male_deep'), ('👨 Male - Normal', 'male_normal'),
        ('👨 Male - Soft', 'male_soft'), ('👨 Male - Young', 'male_young'),
        ('👴 Male - Elder', 'male_elder'),
        ('👩 Female - Deep', 'female_deep'), ('👩 Female - Normal', 'female_normal'),
        ('👩 Female - Soft', 'female_soft'), ('👩 Female - Young', 'female_young'),
        ('👵 Female - Elder', 'female_elder'),
        ('👧 Child - Girl', 'child_girl'),
        ('🎤 Narrator', 'narrator'), ('📺 News Anchor', 'news_anchor'),
        ('🤖 Assistant', 'assistant'),
    ],
    value='all',
    description='Type:',
    style={'description_width': '80px'},
    layout=widgets.Layout(width='49%')
)

quick_voice = widgets.Dropdown(
    options=get_voice_options_by_type('all'),
    description='Voice:',
    style={'description_width': '80px'},
    layout=widgets.Layout(width='100%')
)

def update_voice_options(change):
    quick_voice.options = get_voice_options_by_type(change['new'])
    if quick_voice.options:
        quick_voice.value = quick_voice.options[0][1]

quick_voice_type.observe(update_voice_options, names='value')

quick_style = widgets.Dropdown(
    options=get_style_options(),
    value='natural',
    description='Style:',
    style={'description_width': '80px'},
    layout=widgets.Layout(width='49%')
)

quick_speed = widgets.FloatSlider(
    value=1.0, min=0.5, max=1.5, step=0.05,
    description='Speed:',
    style={'description_width': '80px'},
    layout=widgets.Layout(width='49%'),
    readout_format='.2f'
)

quick_generate_btn = widgets.Button(
    description='🎙️ Generate Speech',
    button_style='success',
    layout=widgets.Layout(width='100%', height='50px'),
    style={'font_weight': 'bold'}
)

quick_status = widgets.HTML("<p style='color: #888; text-align: center;'>Ready to generate...</p>")
quick_output = widgets.Output()

def on_quick_generate(b):
    with quick_output:
        clear_output()
        quick_status.value = "<p style='color: #3498db; text-align: center;'>🔄 Generating audio...</p>"

        try:
            content_type = ContentType(quick_content_type.value)
            style = SpeakingStyle(quick_style.value)

            result = tts_engine.synthesize_with_preset(
                text=quick_text.value,
                voice_preset_id=quick_voice.value,
                content_type=content_type,
                style=style,
                custom_overrides={'speed': quick_speed.value}
            )

            quick_status.value = f"""
            <div style='background: #d4edda; padding: 15px; border-radius: 10px; text-align: center;'>
                <p style='color: #155724; margin: 0;'>✅ Generated successfully!</p>
                <p style='color: #666; margin: 5px 0 0 0; font-size: 0.9em;'>
                    Duration: {result['duration_s']}s | Processing: {result['processing_time']}s
                </p>
            </div>
            """

            display(Audio(result['output_file'], autoplay=True))
            display(FileLink(result['output_file'], result_html_prefix="📥 Download: "))

        except Exception as e:
            quick_status.value = f"<p style='color: #dc3545; text-align: center;'>❌ Error: {str(e)}</p>"

quick_generate_btn.on_click(on_quick_generate)

# Content type info display
quick_info = widgets.HTML("")

def update_content_info(change):
    content = CONTENT_PRESETS.get(ContentType(change['new']))
    if content:
        rec_voices = ", ".join([VOICE_PRESETS[v].name for v in content.recommended_voices[:3] if v in VOICE_PRESETS])
        quick_info.value = f"""
        <div style='background: #f8f9fa; padding: 10px; border-radius: 8px; margin: 10px 0; border-left: 4px solid #3498db;'>
            <b>💡 {content.name}</b><br>
            <small style='color: #666;'>{content.description}</small><br>
            <small style='color: #888;'>Recommended: {rec_voices}</small><br>
            <small style='color: #28a745;'>💡 Tip: {content.tips}</small>
        </div>
        """

quick_content_type.observe(update_content_info, names='value')
update_content_info({'new': 'youtube'})

quick_tab = widgets.VBox([
    widgets.HTML("<h3 style='color: #333;'>⚡ Quick Generate</h3>"),
    quick_text,
    widgets.HBox([quick_content_type, quick_voice_type]),
    quick_voice,
    quick_info,
    widgets.HBox([quick_style, quick_speed]),
    quick_generate_btn,
    quick_status,
    quick_output
], layout=widgets.Layout(padding='20px'))


# ═══════════════════════════════════════════════════════════════════════════════
# TAB 2: ADVANCED STUDIO (Full Control)
# ═══════════════════════════════════════════════════════════════════════════════

adv_text = widgets.Textarea(
    value='Enter your text here for advanced synthesis with complete control over all parameters.',
    layout=widgets.Layout(width='100%', height='150px')
)

# Voice Selection Section
adv_section_voice = widgets.HTML("<h4 style='color: #e94560; border-bottom: 2px solid #e94560; padding-bottom: 5px;'>🎭 Voice Selection</h4>")

adv_use_preset = widgets.Checkbox(value=True, description='Use Voice Preset',
                                    style={'description_width': 'auto'})

adv_voice_preset = widgets.Dropdown(
    options=get_voice_options_by_type('all'),
    description='Preset:',
    style={'description_width': '80px'},
    layout=widgets.Layout(width='100%')
)

adv_locale = widgets.Dropdown(
    options=[('🌍 All Locales', 'all')] + [(f"{loc} ({len(voices)})", loc)
             for loc, voices in sorted(VOICES_BY_LOCALE.items())],
    value='all',
    description='Locale:',
    style={'description_width': '80px'},
    layout=widgets.Layout(width='49%')
)

adv_gender = widgets.Dropdown(
    options=[('All Genders', 'all'), ('👩 Female', 'Female'), ('👨 Male', 'Male')],
    value='all',
    description='Gender:',
    style={'description_width': '80px'},
    layout=widgets.Layout(width='49%')
)

adv_raw_voice = widgets.Dropdown(
    options=[],
    description='Voice:',
    style={'description_width': '80px'},
    layout=widgets.Layout(width='100%'),
    disabled=True
)

def update_raw_voices(change=None):
    voices = EDGE_VOICES
    if adv_locale.value != 'all':
        voices = [v for v in voices if v['Locale'] == adv_locale.value]
    if adv_gender.value != 'all':
        voices = [v for v in voices if v['Gender'] == adv_gender.value]

    emoji_map = {'Female': '👩', 'Male': '👨'}
    options = [(f"{emoji_map.get(v['Gender'], '🎤')} {v['ShortName'].split('-')[-1].replace('Neural', '')} ({v['Locale']})",
                v['ShortName']) for v in voices]
    adv_raw_voice.options = options[:100]
    if options:
        adv_raw_voice.value = options[0][1]

def toggle_voice_mode(change):
    adv_voice_preset.disabled = not change['new']
    adv_raw_voice.disabled = change['new']
    adv_locale.disabled = change['new']
    adv_gender.disabled = change['new']

adv_use_preset.observe(toggle_voice_mode, names='value')
adv_locale.observe(lambda c: update_raw_voices(), names='value')
adv_gender.observe(lambda c: update_raw_voices(), names='value')
update_raw_voices()

# Parameters Section
adv_section_params = widgets.HTML("<h4 style='color: #e94560; border-bottom: 2px solid #e94560; padding-bottom: 5px; margin-top: 20px;'>🎛️ Voice Parameters</h4>")

adv_speed = widgets.FloatSlider(value=1.0, min=0.5, max=2.0, step=0.05, description='Speed:',
                                 style={'description_width': '80px'}, layout=widgets.Layout(width='32%'))
adv_pitch = widgets.FloatSlider(value=1.0, min=0.5, max=1.5, step=0.05, description='Pitch:',
                                 style={'description_width': '80px'}, layout=widgets.Layout(width='32%'))
adv_volume = widgets.FloatSlider(value=1.0, min=0.1, max=2.0, step=0.1, description='Volume:',
                                  style={'description_width': '80px'}, layout=widgets.Layout(width='32%'))

# Preprocessing Section
adv_section_preprocess = widgets.HTML("<h4 style='color: #e94560; border-bottom: 2px solid #e94560; padding-bottom: 5px; margin-top: 20px;'>📝 Text Processing</h4>")

adv_preprocess = widgets.Checkbox(value=True, description='Enable text preprocessing')
adv_expand_abbr = widgets.Checkbox(value=True, description='Expand abbreviations (Dr. → Doctor)')
adv_convert_nums = widgets.Checkbox(value=True, description='Convert numbers (123 → one hundred twenty three)')
adv_convert_time = widgets.Checkbox(value=True, description='Convert time (2:30 → two thirty)')

# Audio Effects Section
adv_section_effects = widgets.HTML("<h4 style='color: #e94560; border-bottom: 2px solid #e94560; padding-bottom: 5px; margin-top: 20px;'>🎨 Audio Effects</h4>")

adv_normalize = widgets.Checkbox(value=False, description='🔊 Normalize audio')
adv_trim_silence = widgets.Checkbox(value=False, description='✂️ Trim silence')
adv_fade_in = widgets.IntSlider(value=0, min=0, max=2000, step=100, description='Fade In (ms):',
                                 style={'description_width': '100px'}, layout=widgets.Layout(width='49%'))
adv_fade_out = widgets.IntSlider(value=0, min=0, max=2000, step=100, description='Fade Out (ms):',
                                  style={'description_width': '100px'}, layout=widgets.Layout(width='49%'))
adv_silence_start = widgets.IntSlider(value=0, min=0, max=3000, step=100, description='Silence Start:',
                                       style={'description_width': '100px'}, layout=widgets.Layout(width='49%'))
adv_silence_end = widgets.IntSlider(value=0, min=0, max=3000, step=100, description='Silence End:',
                                     style={'description_width': '100px'}, layout=widgets.Layout(width='49%'))

# Output Section
adv_section_output = widgets.HTML("<h4 style='color: #e94560; border-bottom: 2px solid #e94560; padding-bottom: 5px; margin-top: 20px;'>📤 Output Settings</h4>")

adv_format = widgets.Dropdown(
    options=[('MP3 (Recommended)', 'mp3'), ('WAV (Lossless)', 'wav'), ('OGG', 'ogg')],
    value='mp3',
    description='Format:',
    style={'description_width': '80px'},
    layout=widgets.Layout(width='49%')
)

# Buttons
adv_generate_btn = widgets.Button(description='🎙️ Generate', button_style='success',
                                   layout=widgets.Layout(width='48%', height='45px'))
adv_preview_btn = widgets.Button(description='👁️ Preview (First 200 chars)', button_style='info',
                                  layout=widgets.Layout(width='48%', height='45px'))

adv_status = widgets.HTML("<p style='color: #888; text-align: center;'>Configure settings and generate...</p>")
adv_output = widgets.Output()

def on_adv_generate(b, preview=False):
    with adv_output:
        clear_output()
        adv_status.value = "<p style='color: #3498db;'>🔄 Processing...</p>"

        try:
            text = adv_text.value
            if preview:
                text = text[:200] + "..." if len(text) > 200 else text

            # Determine voice
            if adv_use_preset.value:
                preset = VOICE_PRESETS[adv_voice_preset.value]
                voice_id = preset.voice_id
            else:
                voice_id = adv_raw_voice.value

            settings = TTSSettings(
                text=text,
                voice_id=voice_id,
                speed=adv_speed.value,
                pitch=adv_pitch.value,
                volume=adv_volume.value,
                preprocess_text=adv_preprocess.value,
                expand_abbreviations=adv_expand_abbr.value,
                convert_numbers=adv_convert_nums.value,
                convert_time=adv_convert_time.value,
                normalize=adv_normalize.value,
                trim_silence=adv_trim_silence.value,
                fade_in_ms=adv_fade_in.value,
                fade_out_ms=adv_fade_out.value,
                silence_start_ms=adv_silence_start.value,
                silence_end_ms=adv_silence_end.value,
                output_format=adv_format.value,
            )

            result = tts_engine.synthesize(settings)

            adv_status.value = f"""
            <div style='background: #d4edda; padding: 10px; border-radius: 8px;'>
                ✅ Generated! Duration: {result['duration_s']}s | Time: {result['processing_time']}s
            </div>
            """

            display(Audio(result['output_file'], autoplay=True))
            display(FileLink(result['output_file'], result_html_prefix="📥 "))

        except Exception as e:
            adv_status.value = f"<p style='color: red;'>❌ Error: {e}</p>"

adv_generate_btn.on_click(lambda b: on_adv_generate(b, preview=False))
adv_preview_btn.on_click(lambda b: on_adv_generate(b, preview=True))

adv_tab = widgets.VBox([
    widgets.HTML("<h3 style='color: #333;'>⚙️ Advanced Studio</h3>"),
    adv_text,
    adv_section_voice,
    adv_use_preset,
    adv_voice_preset,
    widgets.HBox([adv_locale, adv_gender]),
    adv_raw_voice,
    adv_section_params,
    widgets.HBox([adv_speed, adv_pitch, adv_volume]),
    adv_section_preprocess,
    widgets.HBox([adv_preprocess, adv_expand_abbr]),
    widgets.HBox([adv_convert_nums, adv_convert_time]),
    adv_section_effects,
    widgets.HBox([adv_normalize, adv_trim_silence]),
    widgets.HBox([adv_fade_in, adv_fade_out]),
    widgets.HBox([adv_silence_start, adv_silence_end]),
    adv_section_output,
    adv_format,
    widgets.HBox([adv_generate_btn, adv_preview_btn]),
    adv_status,
    adv_output
], layout=widgets.Layout(padding='20px'))


# ═══════════════════════════════════════════════════════════════════════════════
# TAB 3: MULTI-VOICE DIALOG
# ═══════════════════════════════════════════════════════════════════════════════

multi_text = widgets.Textarea(
    value="""Sarah: Welcome to our podcast! Today we have a special guest.
Marcus: Thank you for having me, Sarah. It's great to be here.
Sarah: So tell us about your journey in the industry.
Marcus: Well, it all started about ten years ago when I discovered my passion.""",
    placeholder='Format: VoiceName: Text to speak',
    layout=widgets.Layout(width='100%', height='200px')
)

multi_voice_map_html = widgets.HTML("""
<div style='background: #f0f7ff; padding: 15px; border-radius: 10px; margin: 10px 0;'>
    <b>🎭 Voice Name Mapping:</b><br>
    <div style='display: grid; grid-template-columns: repeat(3, 1fr); gap: 10px; margin-top: 10px;'>
        <span><b>Sarah</b> → Jenny (Female)</span>
        <span><b>Marcus</b> → Guy (Male Deep)</span>
        <span><b>Emma</b> → Sonia (UK Female)</span>
        <span><b>James</b> → Ryan (UK Male)</span>
        <span><b>Olivia</b> → Aria (Female)</span>
        <span><b>Alex</b> → Davis (Male)</span>
    </div>
    <small style='color: #666; display: block; margin-top: 10px;'>
        Or use voice IDs directly: en-US-JennyNeural, en-US-GuyNeural, etc.
    </small>
</div>
""")

multi_voice_map = {
    'sarah': 'en-US-JennyNeural',
    'marcus': 'en-US-GuyNeural',
    'emma': 'en-GB-SoniaNeural',
    'james': 'en-GB-RyanNeural',
    'olivia': 'en-US-AriaNeural',
    'alex': 'en-US-DavisNeural',
    'luna': 'en-US-SaraNeural',
    'ethan': 'en-US-JasonNeural',
    'narrator': 'en-US-GuyNeural',
    'host': 'en-US-JennyNeural',
    'guest': 'en-US-DavisNeural',
}

multi_gap = widgets.IntSlider(value=400, min=0, max=1500, step=50, description='Gap (ms):',
                               style={'description_width': '80px'}, layout=widgets.Layout(width='50%'))

multi_generate_btn = widgets.Button(description='🎭 Generate Multi-Voice', button_style='success',
                                     layout=widgets.Layout(width='100%', height='45px'))

multi_status = widgets.HTML("")
multi_output = widgets.Output()

def on_multi_generate(b):
    with multi_output:
        clear_output()
        multi_status.value = "<p style='color: #3498db;'>🔄 Processing segments...</p>"

        try:
            lines = [l.strip() for l in multi_text.value.split('\n') if l.strip() and ':' in l]
            segments = []

            for line in lines:
                parts = line.split(':', 1)
                voice_name = parts[0].strip().lower()
                text = parts[1].strip()

                # Find voice
                voice_id = multi_voice_map.get(voice_name)
                if not voice_id:
                    # Try direct voice ID
                    if voice_name.endswith('neural'):
                        voice_id = voice_name
                    else:
                        voice_id = 'en-US-JennyNeural'

                segments.append({'text': text, 'voice_id': voice_id})

            if not segments:
                multi_status.value = "<p style='color: red;'>❌ No valid segments found!</p>"
                return

            print(f"🎭 Generating {len(segments)} segments...")

            result = tts_engine.synthesize_multi_voice(segments, gap_ms=multi_gap.value)

            multi_status.value = f"""
            <div style='background: #d4edda; padding: 10px; border-radius: 8px;'>
                ✅ Generated {result['segments']} segments! Total: {result['duration_s']}s
            </div>
            """

            display(Audio(result['output_file'], autoplay=True))
            display(FileLink(result['output_file'], result_html_prefix="📥 "))

        except Exception as e:
            multi_status.value = f"<p style='color: red;'>❌ Error: {e}</p>"

multi_generate_btn.on_click(on_multi_generate)

multi_tab = widgets.VBox([
    widgets.HTML("<h3 style='color: #333;'>🎭 Multi-Voice Dialog</h3><p>Create conversations with multiple voices</p>"),
    multi_text,
    multi_voice_map_html,
    multi_gap,
    multi_generate_btn,
    multi_status,
    multi_output
], layout=widgets.Layout(padding='20px'))


# ═══════════════════════════════════════════════════════════════════════════════
# TAB 4: BATCH PROCESSING
# ═══════════════════════════════════════════════════════════════════════════════

batch_text = widgets.Textarea(
    value="""Welcome to our channel! Don't forget to subscribe.
In this video, we'll explore something amazing.
Let's get started with the tutorial.
Thanks for watching! See you in the next video.""",
    placeholder='One text per line...',
    layout=widgets.Layout(width='100%', height='180px')
)

batch_voice = widgets.Dropdown(
    options=get_voice_options_by_type('all'),
    description='Voice:',
    style={'description_width': '80px'},
    layout=widgets.Layout(width='100%')
)

batch_speed = widgets.FloatSlider(value=1.0, min=0.5, max=1.5, step=0.05, description='Speed:',
                                   style={'description_width': '80px'}, layout=widgets.Layout(width='49%'))
batch_merge = widgets.Checkbox(value=True, description='🔗 Merge into single file', layout=widgets.Layout(width='49%'))
batch_gap = widgets.IntSlider(value=500, min=0, max=2000, step=100, description='Gap (ms):',
                               style={'description_width': '80px'}, layout=widgets.Layout(width='49%'))

batch_generate_btn = widgets.Button(description='🚀 Generate Batch', button_style='success',
                                     layout=widgets.Layout(width='100%', height='45px'))
batch_progress = widgets.IntProgress(value=0, min=0, max=100, description='Progress:',
                                      layout=widgets.Layout(width='100%'))
batch_status = widgets.HTML("")
batch_output = widgets.Output()

def on_batch_generate(b):
    with batch_output:
        clear_output()
        batch_status.value = "<p style='color: #3498db;'>🔄 Processing batch...</p>"

        try:
            texts = [l.strip() for l in batch_text.value.split('\n') if l.strip()]
            if not texts:
                batch_status.value = "<p style='color: red;'>❌ No texts found!</p>"
                return

            batch_progress.max = len(texts)
            batch_progress.value = 0

            preset = VOICE_PRESETS[batch_voice.value]
            settings = TTSSettings(
                text="",
                voice_id=preset.voice_id,
                speed=batch_speed.value,
            )

            def progress_cb(current, total):
                batch_progress.value = current

            result = tts_engine.batch_synthesize(
                texts=texts,
                settings_template=settings,
                merge=batch_merge.value,
                gap_ms=batch_gap.value,
                progress_callback=progress_cb
            )

            batch_status.value = f"<div style='background: #d4edda; padding: 10px; border-radius: 8px;'>✅ Processed {len(texts)} items!</div>"

            if batch_merge.value and 'merged_file' in result:
                display(Audio(result['merged_file'], autoplay=True))
                display(FileLink(result['merged_file'], result_html_prefix="📥 Merged: "))
            else:
                for r in result.get('files', []):
                    display(Audio(r['output_file']))

        except Exception as e:
            batch_status.value = f"<p style='color: red;'>❌ Error: {e}</p>"

batch_generate_btn.on_click(on_batch_generate)

batch_tab = widgets.VBox([
    widgets.HTML("<h3 style='color: #333;'>📦 Batch Processing</h3><p>Generate multiple audio files at once</p>"),
    batch_text,
    batch_voice,
    widgets.HBox([batch_speed, batch_merge]),
    batch_gap,
    batch_generate_btn,
    batch_progress,
    batch_status,
    batch_output
], layout=widgets.Layout(padding='20px'))


# ═══════════════════════════════════════════════════════════════════════════════
# TAB 5: VOICE EXPLORER
# ═══════════════════════════════════════════════════════════════════════════════

exp_filter_type = widgets.Dropdown(
    options=[
        ('🎭 All Types', 'all'),
        ('👨 Male - Deep', 'male_deep'), ('👨 Male - Normal', 'male_normal'),
        ('👨 Male - Soft', 'male_soft'), ('👨 Male - Young', 'male_young'),
        ('👴 Male - Elder', 'male_elder'),
        ('👩 Female - Deep', 'female_deep'), ('👩 Female - Normal', 'female_normal'),
        ('👩 Female - Soft', 'female_soft'), ('👩 Female - Young', 'female_young'),
        ('👵 Female - Elder', 'female_elder'),
        ('👧 Child', 'child_girl'),
        ('🎤 Narrator', 'narrator'), ('📺 News Anchor', 'news_anchor'),
    ],
    value='all',
    description='Filter:',
    style={'description_width': '60px'},
    layout=widgets.Layout(width='40%')
)

exp_test_text = widgets.Text(
    value='Hello! This is a voice test. How does this sound to you?',
    description='Test:',
    style={'description_width': '60px'},
    layout=widgets.Layout(width='58%')
)

exp_output = widgets.Output()

def create_voice_preview_card(preset_id, preset):
    """Create a preview card for a voice"""
    emoji = "👩" if preset.gender == "Female" else "👨"
    if "child" in preset.voice_type.value:
        emoji = "👧" if preset.gender == "Female" else "👦"
    elif "elder" in preset.voice_type.value:
        emoji = "👵" if preset.gender == "Female" else "👴"

    card = widgets.VBox([
        widgets.HTML(f"""
        <div style='border: 2px solid #e0e0e0; padding: 15px; border-radius: 12px;
                    background: linear-gradient(145deg, #ffffff, #f5f5f5); min-width: 200px;'>
            <div style='font-size: 2em; text-align: center;'>{emoji}</div>
            <div style='text-align: center; font-weight: bold; color: #333;'>{preset.name}</div>
            <div style='text-align: center; color: #888; font-size: 0.85em;'>{preset.locale}</div>
            <div style='text-align: center; color: #666; font-size: 0.8em; margin-top: 5px;'>
                {preset.description[:50]}...
            </div>
            <div style='margin-top: 8px;'>
                {''.join([f"<span style='background: #e0e0e0; padding: 2px 6px; border-radius: 10px; font-size: 0.7em; margin: 2px;'>{tag}</span>" for tag in preset.tags[:3]])}
            </div>
        </div>
        """),
    ], layout=widgets.Layout(margin='5px'))

    test_btn = widgets.Button(description='▶️ Test', button_style='info', layout=widgets.Layout(width='100%'))

    def test_voice(b):
        with exp_output:
            print(f"🎤 Testing: {preset.name}")
            result = tts_engine.synthesize_with_preset(exp_test_text.value, preset_id)
            display(Audio(result['output_file'], autoplay=True))

    test_btn.on_click(test_voice)
    card.children = list(card.children) + [test_btn]

    return card

def display_voices(b=None):
    with exp_output:
        clear_output()

        filter_type = exp_filter_type.value

        presets = [(k, v) for k, v in VOICE_PRESETS.items()
                   if filter_type == 'all' or v.voice_type.value == filter_type]

        print(f"📊 Showing {len(presets)} voice presets\n")

        cards = [create_voice_preview_card(k, v) for k, v in presets]

        # Display in rows of 3
        for i in range(0, len(cards), 3):
            display(widgets.HBox(cards[i:i+3]))

exp_filter_type.observe(lambda c: display_voices(), names='value')

exp_tab = widgets.VBox([
    widgets.HTML("<h3 style='color: #333;'>🔍 Voice Explorer</h3><p>Browse and test all voice presets</p>"),
    widgets.HBox([exp_filter_type, exp_test_text]),
    widgets.HTML("<hr>"),
    exp_output
], layout=widgets.Layout(padding='20px'))

# Initial display
display_voices()


# ═══════════════════════════════════════════════════════════════════════════════
# TAB 6: CONTENT PRESETS INFO
# ═══════════════════════════════════════════════════════════════════════════════

def create_content_card(content_type, preset):
    icons = {
        'youtube': '📺', 'podcast': '🎙️', 'audiobook': '📚', 'documentary': '🎬',
        'commercial': '📢', 'elearning': '🎓', 'meditation': '🧘', 'news': '📰',
        'gaming': '🎮', 'ivr': '📞', 'social': '📱', 'presentation': '📊',
        'storytelling': '📖', 'voiceover': '🎤', 'custom': '⚙️'
    }
    icon = icons.get(content_type.value, '🎯')

    rec_voices = []
    for v in preset.recommended_voices[:3]:
        if v in VOICE_PRESETS:
            rec_voices.append(VOICE_PRESETS[v].name)

    return widgets.HTML(f"""
    <div style='border: 2px solid #3498db; padding: 20px; border-radius: 15px;
                margin: 10px; background: #f8f9fa; min-width: 280px;'>
        <div style='font-size: 2.5em; text-align: center;'>{icon}</div>
        <h3 style='text-align: center; color: #2c3e50; margin: 10px 0;'>{preset.name}</h3>
        <p style='color: #666; text-align: center; font-size: 0.9em;'>{preset.description}</p>

        <div style='background: white; padding: 10px; border-radius: 8px; margin: 10px 0;'>
            <div style='display: grid; grid-template-columns: 1fr 1fr; gap: 5px; font-size: 0.85em;'>
                <span>⚡ Speed: {preset.default_speed}x</span>
                <span>🎚️ Pitch: {preset.default_pitch}x</span>
                <span>⏸️ Pause: {preset.pause_between_sentences}ms</span>
                <span>🎨 Style: {preset.style.value}</span>
            </div>
        </div>

        <div style='font-size: 0.85em; color: #666;'>
            <b>🎭 Recommended:</b> {', '.join(rec_voices)}
        </div>

        <div style='background: #e8f4f8; padding: 8px; border-radius: 5px; margin-top: 10px; font-size: 0.85em;'>
            💡 <em>{preset.tips}</em>
        </div>
    </div>
    """)

content_cards = [create_content_card(ct, preset) for ct, preset in CONTENT_PRESETS.items()]

content_tab = widgets.VBox([
    widgets.HTML("<h3 style='color: #333;'>📹 Content Type Presets</h3><p>Pre-configured settings for different content types</p>"),
    widgets.HBox(content_cards[:3]),
    widgets.HBox(content_cards[3:6]),
    widgets.HBox(content_cards[6:9]),
    widgets.HBox(content_cards[9:12]),
    widgets.HBox(content_cards[12:]),
], layout=widgets.Layout(padding='20px', overflow_x='auto'))


# ═══════════════════════════════════════════════════════════════════════════════
# ASSEMBLE MAIN UI
# ═══════════════════════════════════════════════════════════════════════════════

tabs = widgets.Tab(children=[
    quick_tab,
    adv_tab,
    multi_tab,
    batch_tab,
    exp_tab,
    content_tab,
])

tabs.set_title(0, '⚡ Quick')
tabs.set_title(1, '⚙️ Advanced')
tabs.set_title(2, '🎭 Multi-Voice')
tabs.set_title(3, '📦 Batch')
tabs.set_title(4, '🔍 Explorer')
tabs.set_title(5, '📹 Presets')

# Footer
footer = widgets.HTML("""
<div style='text-align: center; padding: 15px; margin-top: 20px; color: #888; font-size: 0.9em;'>
    🎙️ Professional TTS Studio | CPU Optimized | Unlimited Generation<br>
    <small>Powered by Edge TTS, Google TTS | All voices are free to use</small>
</div>
""")

main_ui = widgets.VBox([
    header_html,
    tabs,
    footer
], layout=widgets.Layout(max_width='1000px', margin='0 auto'))

display(main_ui)

print("\n" + "═"*60)
print("✅ PROFESSIONAL TTS STUDIO READY!")
print("═"*60)

In [ ]:
#@title 🛠️ **CELL 4: Utilities & Export Tools**
#@markdown ### Additional tools for managing files and history

util_header = widgets.HTML("""
<div style='background: #34495e; padding: 20px; border-radius: 15px; margin-bottom: 15px;'>
    <h2 style='color: white; margin: 0; text-align: center;'>🛠️ Utilities & Tools</h2>
</div>
""")

# ═══════ FILE MANAGER ═══════
file_output = widgets.Output()

def list_files(b=None):
    with file_output:
        clear_output()
        print("📁 Generated Audio Files:\n")

        files = list(DIRS['audio'].glob('*.mp3')) + list(DIRS['audio'].glob('*.wav'))
        files.sort(key=lambda x: x.stat().st_mtime, reverse=True)

        if not files:
            print("No files found.")
            return

        for f in files[:20]:
            size_kb = f.stat().st_size / 1024
            mod_time = datetime.fromtimestamp(f.stat().st_mtime).strftime('%Y-%m-%d %H:%M')
            print(f"  📄 {f.name} ({size_kb:.1f} KB) - {mod_time}")
            display(FileLink(str(f), result_html_prefix="     📥 "))

def clear_files(b):
    with file_output:
        clear_output()
        for f in DIRS['audio'].glob('*'):
            f.unlink()
        for f in DIRS['temp'].glob('*'):
            f.unlink()
        print("✅ All files cleared!")

def download_all(b):
    with file_output:
        clear_output()
        import shutil
        zip_path = BASE_DIR / "all_audio.zip"
        shutil.make_archive(str(BASE_DIR / "all_audio"), 'zip', str(DIRS['audio']))
        print("✅ Created zip archive!")
        display(FileLink(str(zip_path), result_html_prefix="📥 Download All: "))

list_btn = widgets.Button(description='📂 List Files', button_style='info')
clear_btn = widgets.Button(description='🗑️ Clear All', button_style='danger')
download_btn = widgets.Button(description='📦 Download All', button_style='success')

list_btn.on_click(list_files)
clear_btn.on_click(clear_files)
download_btn.on_click(download_all)

# ═══════ STATS ═══════
def show_stats(b=None):
    with file_output:
        clear_output()

        audio_files = list(DIRS['audio'].glob('*'))
        total_size = sum(f.stat().st_size for f in audio_files) / (1024 * 1024)

        print("📊 Session Statistics:\n")
        print(f"   🎵 Audio files: {len(audio_files)}")
        print(f"   💾 Total size: {total_size:.2f} MB")
        print(f"   📝 History entries: {len(tts_engine.history)}")
        print(f"   🎭 Voice presets: {len(VOICE_PRESETS)}")
        print(f"   📹 Content presets: {len(CONTENT_PRESETS)}")
        print(f"   🌍 Available voices: {len(EDGE_VOICES)}")

stats_btn = widgets.Button(description='📊 Show Stats', button_style='info')
stats_btn.on_click(show_stats)

util_ui = widgets.VBox([
    util_header,
    widgets.HBox([list_btn, clear_btn, download_btn, stats_btn]),
    widgets.HTML("<hr>"),
    file_output
], layout=widgets.Layout(padding='15px'))

display(util_ui)
list_files()